# In this notebook we look at using SQLite databases using Python and Pandas

## References:
* https://www.dataquest.io/blog/python-pandas-databases/

In [2]:
import pandas as pd
import sqlite3

Once you have a connection object, you can create a cursor object that let's you execute SQL queries. 
* Remember to close the cursor and connection objects when not in use since they lock the SQLite database and it might not be able to be updated
* The results are given in an array of tuples

In [3]:
# Connect to the database
conn = sqlite3.connect("data/flights.db")

# Create a cursor object
cur = conn.cursor()

# Execute query
cur.execute("SELECT * FROM airlines LIMIT 5;")

# Fetch the results
results = cur.fetchall()
print(results)

# Close the connection
cur.close()
conn.close()

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'), (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'), (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'), (3, '4', '2 Sqn No 1 Elementary Flying Training School', '\\N', None, 'WYT', None, 'United Kingdom', 'N'), (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]


## Using Pandas

In [6]:
# Create the connection object
conn = sqlite3.connect("data/flights.db")

# Read the data from an SQL query
query = "SELECT * FROM airlines LIMIT 10;"

df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

df.head(n=5)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


## Modifying the database rows
* We can insert, update or delete rows
* You need to specify the values into columns
* You need to commit a transaction to do 1 or more queries to do all the changes to the database at once

In [18]:
# Create the connection object
conn = sqlite3.connect("data/flights.db")

# Create the cursor object
cur = conn.cursor()

# Insert query
query = "INSERT INTO airlines VALUES (6048, 19846, 'Test flight', '', '', null, null, null, 'Y')"

# Execute the query
cur.execute(query)

# Commit all the changes
conn.commit()

print(pd.read_sql_query("SELECT * FROM airlines where id=19846;", conn))

# Close the Cursor and the connection
cur.close()
conn.close()

index     id         name alias iata  icao callsign country active
0   6048  19846  Test flight             None     None    None      Y
1   6048  19846  Test flight             None     None    None      Y
2   6048  19846  Test flight             None     None    None      Y


In [20]:
# To delete a row, we do sometying similar

# Create the connection object
conn = sqlite3.connect("data/flights.db")

# Create the cursor object
cur = conn.cursor()

# Insert query
values = (19846, ) # has to be a tuple?
query = "DELETE FROM airlines WHERE id=?"

# Execute the query
cur.execute(query,values)

# Commit all the changes
conn.commit()

print(pd.read_sql_query("SELECT * FROM airlines where id=19846;", conn))

# Close the Cursor and the connection
cur.close()
conn.close()

Empty DataFrame
Columns: [index, id, name, alias, iata, icao, callsign, country, active]
Index: []


## Creating a table with pandas
* You could also add a column to a table by creating a new pandas DF with the new column and replace the table in the database

In [25]:
from datetime import datetime

df = pd.DataFrame(
    [[1, datetime(2016, 9, 29, 0, 0) ,
    datetime(2016, 9, 29, 12, 0), 'T1', 1]],
    columns=["id", "departure", "arrival", "number", "route_id"])

# Create the connection object
conn = sqlite3.connect("data/flights.db")

# Sendthe table to the SQLite database
df.to_sql("daily_flights", conn, if_exists="replace")

# Read the new table
query = "SELECT * FROM daily_flights"
df_new = pd.read_sql_query(query, conn)

# Close connection
conn.close()

df_new.head()

,index,id,departure,arrival,number,route_id
0,0,1,2016-09-29 00:00:00,2016-09-29 12:00:00,T1,1
